<h2><a href="https://leetcode.com/problems/grand-slam-titles">1932. Grand Slam Titles</a></h2><h3>Medium</h3><hr><p>Table: <code>Players</code></p>

<pre>
+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| player_id      | int     |
| player_name    | varchar |
+----------------+---------+
player_id is the primary key (column with unique values) for this table.
Each row in this table contains the name and the ID of a tennis player.
</pre>

<p>&nbsp;</p>

<p>Table: <code>Championships</code></p>

<pre>
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| year          | int     |
| Wimbledon     | int     |
| Fr_open       | int     |
| US_open       | int     |
| Au_open       | int     |
+---------------+---------+
year is the primary key (column with unique values) for this table.
Each row of this table contains the IDs of the players who won one each tennis tournament of the grand slam.
</pre>

<p>&nbsp;</p>

<p>Write a solution to report the number of grand slam tournaments won by each player. Do not include the players who did not win any tournament.</p>

<p>Return the result table in <strong>any order</strong>.</p>

<p>The result format is in the following example.</p>

<p>&nbsp;</p>
<p><strong class="example">Example 1:</strong></p>

<pre>
<strong>Input:</strong> 
Players table:
+-----------+-------------+
| player_id | player_name |
+-----------+-------------+
| 1         | Nadal       |
| 2         | Federer     |
| 3         | Novak       |
+-----------+-------------+
Championships table:
+------+-----------+---------+---------+---------+
| year | Wimbledon | Fr_open | US_open | Au_open |
+------+-----------+---------+---------+---------+
| 2018 | 1         | 1       | 1       | 1       |
| 2019 | 1         | 1       | 2       | 2       |
| 2020 | 2         | 1       | 2       | 2       |
+------+-----------+---------+---------+---------+
<strong>Output:</strong> 
+-----------+-------------+-------------------+
| player_id | player_name | grand_slams_count |
+-----------+-------------+-------------------+
| 2         | Federer     | 5                 |
| 1         | Nadal       | 7                 |
+-----------+-------------+-------------------+
<strong>Explanation:</strong> 
Player 1 (Nadal) won 7 titles: Wimbledon (2018, 2019), Fr_open (2018, 2019, 2020), US_open (2018), and Au_open (2018).
Player 2 (Federer) won 5 titles: Wimbledon (2020), US_open (2019, 2020), and Au_open (2019, 2020).
Player 3 (Novak) did not win anything, we did not include them in the result table.
</pre>

In [2]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("GrandSlamTitles").getOrCreate()

# Create the Players DataFrame
players_data = [
    (1, "Nadal"),
    (2, "Federer"),
    (3, "Novak")
]
players_columns = ["player_id", "player_name"]
players_df = spark.createDataFrame(players_data, players_columns)

# Create the Championships DataFrame
championships_data = [
    (2018, 1, 1, 1, 1),
    (2019, 1, 1, 2, 2),
    (2020, 2, 1, 2, 2)
]
championships_columns = ["year", "Wimbledon", "Fr_open", "US_open", "Au_open"]
championships_df = spark.createDataFrame(championships_data, championships_columns)

# Register the DataFrames as temporary views for SQL querying
players_df.createOrReplaceTempView("Players")
championships_df.createOrReplaceTempView("Championships")

players_df.show()
championships_df.show()

+---------+-----------+
|player_id|player_name|
+---------+-----------+
|        1|      Nadal|
|        2|    Federer|
|        3|      Novak|
+---------+-----------+

+----+---------+-------+-------+-------+
|year|Wimbledon|Fr_open|US_open|Au_open|
+----+---------+-------+-------+-------+
|2018|        1|      1|      1|      1|
|2019|        1|      1|      2|      2|
|2020|        2|      1|      2|      2|
+----+---------+-------+-------+-------+



In [7]:
spark.sql("""
    WITH unioned_data AS (
            SELECT Wimbledon AS player_id, count(*) as championship_count
            FROM Championships
            GROUP BY Wimbledon 
            UNION ALL
            SELECT Fr_open AS player_id, count(*) as championship_count
            FROM Championships
            GROUP BY Fr_open 
            UNION ALL
            SELECT US_open AS player_id, count(*) as championship_count
            FROM Championships
            GROUP BY US_open 
            UNION ALL
            SELECT Au_open AS player_id, count(*) as championship_count
            FROM Championships
            GROUP BY Au_open 
        )
          
    SELECT 
          t.player_id, p.player_name, sum(championship_count) as grand_slams_count
          FROM unioned_data t
          INNER JOIN Players p on t.player_id = p.player_id
          GROUP BY t.player_id, p.player_name
""").show()

+---------+-----------+-----------------+
|player_id|player_name|grand_slams_count|
+---------+-----------+-----------------+
|        1|      Nadal|                7|
|        2|    Federer|                5|
+---------+-----------+-----------------+



25/02/23 17:33:36 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 934507 ms exceeds timeout 120000 ms
25/02/23 17:33:36 WARN SparkContext: Killing executors is not supported by current scheduler.
25/02/23 17:33:37 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$